# Aula 1

In [ ]:
!pip install litellm

In [ ]:
from litellm import completion

In [ ]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [ ]:
# Função para chamar a API com o histórico de mensagens
def call_groq_api(messages, model="groq/llama3-groq-70b-8192-tool-use-preview"):
    response = completion(
        model=model,
        messages=messages,
        api_key=GROQ_API_KEY,
    )
    return response.choices[0].message.content


In [ ]:
# Função para iniciar o chat, mantendo o histórico
def chat():
    print("Iniciando chat com o modelo. Digite 'sair' para encerrar.")


    # Histórico de mensagens
    messages = [{"role": "system", "content": """
    Você é o chatbot da Pizza Turtle, uma pizzaria com um ambiente acolhedor, divertido e familiar.
    Sua personalidade deve ser amigável, leve e entusiasmada, com uma linguagem descontraída.
    Sempre que possível, use expressões simpáticas e emojis de tartaruga 🐢 e pizza 🍕 de forma moderada para reforçar a identidade da marca.
    Você deve conhecer o cardápio, incluindo os ingredientes, tamanhos, opções de personalização, e promoções atuais.
    Sugira sabores populares e acompanhamentos, especialmente para clientes indecisos, e sempre ofereça opções de personalização, explicando com clareza as escolhas de massa, tamanho e ingredientes extras. Facilite o processo de pedido de forma direta e simpática, e ao informar o tempo de entrega ou retirada, mantenha a honestidade e mostre empatia se houver algum atraso. Responda perguntas sobre ingredientes, alérgenos, opções vegetarianas/veganas e formas de pagamento de maneira clara e informativa. Após finalizar o pedido, acompanhe o cliente com uma despedida simpática e pergunte, se apropriado, pelo feedback sobre a experiência. Sua linguagem deve ser simples e direta, evitando jargões ou formalidades.
    """}]


    while True:
        user_message = input("Você: ")
        if user_message.lower() == "sair":
            print("Encerrando chat. Até a próxima!")
            break


        # Adicionar a mensagem do usuário ao histórico
        messages.append({"role": "user", "content": user_message})


        # Chamar a API com o histórico completo
        model_response = call_groq_api(messages)


        # Adicionar a resposta do modelo ao histórico
        messages.append({"role": "assistant", "content": model_response})


        # Exibir a resposta do assistente
        print(f"Assistente: {model_response}")


In [ ]:
chat()

# Aula 2

In [13]:
import requests
import json

def verificar_rua_cep(cep):
    # Lista de ruas atendidas
    ruas_atendidas = ["Rua Vergueiro", "Avenida Paulista", "Rua Augusta"]

    # URL da API ViaCEP para consulta de endereço
    url = f"https://viacep.com.br/ws/{cep}/json/"

    try:
        response = requests.get(url)
        response.raise_for_status()
        endereco = response.json()

        # Verifica se o CEP é válido e possui logradouro
        if 'logradouro' in endereco:
            logradouro = endereco['logradouro']
            if logradouro in ruas_atendidas:
                return f"A rua '{logradouro}' está na lista de ruas atendidas."
            else:
                return f"A rua '{logradouro}' NÃO está na lista de ruas atendidas."
        else:
            return "CEP inválido ou sem logradouro associado."

    except requests.RequestException as e:
        return f"Erro ao consultar o CEP: {e}"




In [14]:
# Exemplo de uso
cep = "01311000"  # Insira o CEP desejado - Avenida Paulista
cep = "05436100" # Beco do Batman
print(verificar_rua_cep(cep))

A rua 'Rua Gonçalo Afonso' NÃO está na lista de ruas atendidas.


In [15]:
tools = [
        {
            "type": "function",
            "function": {
                "name": "verificar_rua_cep",
                "description": "Retorna se o CEP corresponde a uma rua atendida",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "cep": {
                            "type": "string",
                            "description": "CEP do endereço",
                        },
                    },
                    "required": ["cep"],
                },
            }

        }
]

In [18]:
# Função para chamar a API com o histórico de mensagens
def call_groq_api(messages, model="groq/llama3-groq-70b-8192-tool-use-preview"):
    global tools
    response = completion(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        api_key=GROQ_API_KEY,
    )
    resposta_texto = response.choices[0].message
    chamada_ferramentas = resposta_texto.tool_calls
    if chamada_ferramentas:
      available_functions = {
        "verificar_rua_cep": verificar_rua_cep,
      }
      for tool_call in chamada_ferramentas:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            cep=function_args.get("cep"),
        )
        return function_response

    else:
      return resposta_texto.content


In [19]:
chat()

Iniciando chat com o modelo. Digite 'sair' para encerrar.
Você: Vocês atendem no CEP 01311000?
Assistente: A rua 'Avenida Paulista' está na lista de ruas atendidas.
Você: Sair
Encerrando chat. Até a próxima!
